In [1]:
# source: https://github.com/karmus89/blockchain-a-z
!pip install Faker
import datetime
import hashlib
import json
import sys
from faker import Faker

class Blockchain:

    def __init__(self):
        
        self.chain = []
        self.create_block(proof=1, previous_hash='0', record = {'username': '', 'name': '', 'sex': '', 'address': '', 'mail': '', 'birthdate': ''})

    def create_block(self, proof, previous_hash,record):
        "Create a block."
        block = {
            'index': len(self.chain),
            'timestamp': str(datetime.datetime.now()),
            'proof': proof,
            'previous_hash': previous_hash,
            'record': record,
        }
        self.chain.append(block)
        
        return block

    def get_previous_block(self):
        
        return self.chain[-1]

    def hash_proof(self, previous_proof, next_proof):
        "Calculate the SHA256-hash"
        
        return (hashlib
                .sha256(str(next_proof**2 - previous_proof**2).encode())
                .hexdigest())

    def hash_block(self, block):
        "Calculate the SHA256-hash for a block."

        encoded_block = json.dumps(block, sort_keys=True, default=str).encode()
        
        return hashlib.sha256(encoded_block).hexdigest()

    def proof_of_work(self, previous_proof):
        "Calculate a new proof related to the previous block."

        next_proof = 1
        check_proof = False

        while check_proof is False:

            if self.hash_proof(previous_proof, next_proof)[:4] == '0000':
                check_proof = True

            else:
                next_proof += 1

        return next_proof

    def is_chain_valid(self, chain):
        "Validate that block and proof hashes are correct across the chain."

        for i in range(len(chain)):

            if i == 0:
                continue

            if chain[i]['previous_hash'] != self.hash_block(chain[i-1]):
                return False

            previous_proof = chain[i-1]['proof']
            next_proof = chain[i]['proof']

            if self.hash_proof(previous_proof, next_proof)[:4] != '0000':
                
                return False

        return True

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import threading
import requests

from flask import Flask, request, jsonify
from werkzeug.serving import run_simple


class BlockchainApp:

    def __init__(self, host='localhost', port=5000, chain=Blockchain):

        self.host = host
        self.port = port
        self.chain = chain()

        self.host_url = f'http://{self.host}:{self.port}/'
        
        self.app = Flask(__name__)
        self.add_api_endpoints()
        
        self.thread = threading.Thread(
            target=run_simple,
            kwargs={
                'hostname': self.host,
                'port': self.port,
                'application': self.app}
        )

    def __enter__(self):

        self.start()

        return self

    def __exit__(self, *args):

        self.stop()
        
    def add_api_endpoints(self):
        "Add API endpoints to the Flask WebApp."
        
        self.app.add_url_rule(
            rule='/blocks',
            view_func=self.api_blocks,
            methods=['GET', 'POST']
        )
        #5.d
        self.app.add_url_rule(
            rule='/blocks/validate',
            view_func=self.api_validate,
        )
        self.app.add_url_rule(
            rule='/shutdown',
            view_func=self.api_shutdown,
        )
        #5.b
        self.app.add_url_rule(
            rule='/male_names',
            view_func=self.api_male_names,
            methods=['GET']
        )
        #5.c
        self.app.add_url_rule(
            rule='/female_names',
            view_func=self.api_female_names,
            methods=['GET']
        )
        #5.e
        self.app.add_url_rule(
            rule='/overwrite_block',
            view_func=self.api_overwrite_block,
            methods=['POST']
        )
    #5.a
    def api_blocks(self):
        "Either retrieve the node's current chain or post a new block to the chain."

        if request.method == 'POST':

            prev_block = self.chain.get_previous_block()
            prev_hash = self.chain.hash_block(prev_block)
            prev_proof = prev_block['proof']

            proof = self.chain.proof_of_work(prev_proof)

            
            fake = Faker(locale='en_US')
            record= fake.simple_profile()
            record['birthdate'] =  str(record['birthdate'])

            block = self.chain.create_block(proof, prev_hash,record)

            response = {'message': 'Congratulations, you just mined a Block!',
                        'block': block}

            return (jsonify(response), 200)

        if request.method == 'GET':

            response = {'chain': self.chain.chain,
                        'length': len(self.chain.chain)}

            return (jsonify(response), 200)

    #5.b
    def api_male_names(self):
      "Retrieve all male names from the blockchain."

      male_names = []
      for block in self.chain.chain:
          if 'record' in block:
              if block['record']['sex'] == 'M':
                  male_names.append(block['record']['name'])

      response = {'male_names': male_names}

      return (jsonify(response), 200)



    #5.c
    def api_female_names(self):
      "Retrieve all female names from the blockchain."
      
      female_names = []
      for block in self.chain.chain:
          if 'record' in block:
              if block['record']['sex'] == 'F':
                  female_names.append(block['record']['username'])
      
      response = {'female_names': female_names}
      
      return (jsonify(response), 200)

    #5.d
    def api_validate(self):
        "Validate the chain"

        if self.chain.is_chain_valid(self.chain.chain):

            response = {'message': 'Chain is valid.',
                        'valid': True}
            return (jsonify(response), 200)

        else:

            response = {'message': 'Chain is not valid!',
                        'valid': False}
            return (jsonify(response), 500)

    #5.e
    def api_overwrite_block(self):
        "Overwrite the third block's record field with a new profile record generated using faker library."

        if len(self.chain.chain) < 3:
            response = {'message': 'There are less than 3 blocks in the chain.'}
            return (jsonify(response), 500)

        # create a new profile record using the faker library
        fake = Faker(locale='en_US')
        new_record = fake.simple_profile()

        # update the record field of the 3rd block in the chain with the new record
        blockchain_app.chain.chain[2]['record'] = new_record
        response = {'message': '3rd record with index:2 has been updated with different details'}

        return (jsonify(response), 200)



    def api_shutdown(self):
        "Shutdown the Flask WebApp"

        # depricated
        # request.environ.get('werkzeug.server.shutdown')()

        return jsonify({'message': 'Shutting down'}), 200

    def start(self):
        "Start the Flask-based Blockchain WebApp."

        self.thread.start()

    def stop(self):
        "Stop the Flask-based Blockchain WebApp."

        if self.thread.is_alive():

            return requests.get(f'{self.host_url}/shutdown')

In [3]:
import requests
import json
import time

with BlockchainApp() as blockchain_app:

    # wait for server thread ready
    time.sleep(5)
    
    print("-------------------------------------------------#5.a-------------------------------------------------")
    # mine 20 blocks
    for x in range(19):

        response = requests.post(f'{blockchain_app.host_url}/blocks')

    response = requests.get(f'{blockchain_app.host_url}/blocks')

    print(json.dumps(response.json(), indent=2))
    print("-------------------------------------------------#5.b-------------------------------------------------")
    # get male names from the blockchain
    male_names_response = requests.get(f'{blockchain_app.host_url}/male_names')
    male_names = male_names_response.json()['male_names']
    
    # print male names
    print("Male Names:")
    for name in male_names:
        print(name)
    print("-------------------------------------------------#5.c-------------------------------------------------")
    # get female usernames from the blockchain
    female_names_response = requests.get(f'{blockchain_app.host_url}/female_names')
    female_names = female_names_response.json()['female_names']
    
    # print female usernames
    print("Female usernames:")
    for name in female_names:
        print(name)
    print("-------------------------------------------------Validating before alteration-------------------------------------------------")   
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate') 
    if response.json()['valid']:
        print('Blockchain is valid.')
    else:
        print('Blockchain is not valid.')
    print("-------------------------------------------------Alteration-------------------------------------------------")   
    response = requests.post(f'{blockchain_app.host_url}/overwrite_block')
    print("Chain={}".format(json.dumps(response.json(), indent=2)))
    print("-------------------------------------------------Printing current Blockchain-------------------------------------------------")   
    # get the current blockchain
    response = requests.get(f'{blockchain_app.host_url}/blocks')
    # print the blockchain
    print(json.dumps(response.json(), indent=2))
    print("-------------------------------------------------Validating after alteration-------------------------------------------------")   
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate') 
    if response.json()['valid']:
        print('Blockchain is valid.')
    else:
        print('Blockchain is not valid.')


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:57] "POST /blocks HTTP/1.1" 200 -


-------------------------------------------------#5.a-------------------------------------------------


INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:57] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:57] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:57] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:58] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:58] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:58] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:58] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:58] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:58] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:59] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:45:59] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:46:00] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - 

{
  "chain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "record": {
        "address": "",
        "birthdate": "",
        "mail": "",
        "name": "",
        "sex": "",
        "username": ""
      },
      "timestamp": "2023-04-15 02:45:52.342913"
    },
    {
      "index": 1,
      "previous_hash": "c5f12b2bae3e74d8908942c51ee8298fbe8ecc7302ec983c0ce2af6b3d9a4032",
      "proof": 533,
      "record": {
        "address": "380 Fisher Springs Suite 034\nDonaldhaven, FL 50775",
        "birthdate": "1974-06-30",
        "mail": "benjamin82@yahoo.com",
        "name": "Randy Russo",
        "sex": "M",
        "username": "ahunter"
      },
      "timestamp": "2023-04-15 02:45:57.419283"
    },
    {
      "index": 2,
      "previous_hash": "a81381ced2671a1207343fa14c94ca5f479e5f5306168577f663275c5bc806dd",
      "proof": 45293,
      "record": {
        "address": "62262 Cuevas Lock\nJasminehaven, MS 13255",
        "birthdate": "1966-04-07",
  